<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : GeoPandas - Projeções.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import pyproj

## Projeção

###  Intro

Como vimos anteriormente, cada figura geométrica em `Geopandas` representa um objeto que requer um [sistema de coordenadas](https://www.ibm.com/docs/en/informix-servers/12.10?topic=data-geographic-coordinate-system) para relacioná-lo a um local específico na Terra. Embora o sistema mais comum seja aquele que usa a latitude e a longitude para indicar a posição do objeto, existem muitos outros.

As [Projeções](https://www.esri.com/arcgis-blog/products/arcgis-pro/mapping/gcs_vs_pcs/) tentam representar a superfície da Terra ou uma porção da Terra em uma folha de papel plana ou em uma tela de computador. Então, um [sistema de referência de coordenadas (CRS)](https://docs.qgis.org/3.16/en/docs/gentle_gis_introduction/coordinate_reference_systems.html) define, com a ajuda de coordenadas, como o mapa bidimensional projetado está relacionado a lugares reais na terra.

Em outras palavras, o [Coordinate Reference System (CRS)](https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/intro-to-coordinate-reference-systems/) informa ao `Python` a localização geográfica real onde os objetos representados pelas formas geométricas estão localizados.

Por outro lado, é importante que todos os objetos sejam expressos com o mesmo CRS, para que as operações entre eles possam ser realizadas. Como às vezes isso não acontece, temos que realizar uma reprojeção de alguns objetos, ou seja, alterar seu sistema de coordenadas, para unificar o CRS de todos os objetos.

Diferentes projeções cartográficas são utilizadas dependendo do tipo de mapa a ser criado, uma vez que existem certas projeções que se adaptam melhor a alguns usos específicos do que a outros, a algumas áreas ou outras. Por exemplo, uma projeção que representa com precisão a forma dos continentes [distorce](https://www.nationalgeographic.com/culture/article/all-over-the-map-mental-mapping-misconceptions) seus tamanhos relativos. Lembre-se de que toda projeção é uma representação da realidade.

###  CRS

Existem vários [CRSs](https://datacarpentry.org/organization-geospatial/03-crs/), dependendo das necessidades de geolocalização. Os [CRS](https://www.w3.org/2015/spatial/wiki/Coordinate_Reference_Systems) são identificados por um código denominado [EPSG](https://epsg.io/). Por exemplo, o sistema de coordenadas que usa latitude e longitude é [CRS WGS 84](https://epsg.io/4326), que é representado por EPSG 4326. Uma lista atualizada é publicada no portal [EPSG](https://epsg.org/home.html).


Exemplos de CRS:

- WGS 84. 

Usa a latitude e a longitude.

<div>
    <div class = "mapa">
        <img src='img/latitude-e-longitude.jpeg' alt="Latitud y Longitud" width=50% height=40%>
        <p><i>Latitude e Longitude</i></p>
    </div>
</div>      

### UTM Universal Transverse Mercator.
    
O [Universal Transverse Mercator (UTM)](https://gisgeography.com/utm-universal-transverse-mercator-projection/) é representado por um mapa retangular. Podemos nos lembrar dos mapas que usamos na escola para representar o planeta. Essa projeção achata a esfera da Terra, dividindo-a em $60$ zonas de norte a sul, cada uma com $6$ graus.
Então, para identificar um lugar, você precisa de um número para a localização norte-sul, outro para a posição leste-oeste (ambos positivos) e a zona a que pertence. É útil para mapear pequenas áreas.

<div>
    <div class = "mapa">
        <img src='img/UTM1.PNG' alt="Esfera terrestre dividida en franjas de 6 grados" width=25% height=40%>
        <p><i>Esfera terrestre dividida en franjas de 6 grados</i></p>
    </div>
</div>

<div>
    <div class = "mapa">
        <img src='img/UTM2.PNG' alt="Mapa rectangular" width=25% height=40%>
        <p><i>Mapa retangular</i></p>
    </div>
</div>

<div>
    <div class = "mapa">
        <img src='img/UTM3.PNG' alt="Coordenadas de Lima, con CRS WGS 84 y UTM" width=35% height=80%>
        <p><i>Coordenadas de Lima, com CRS WGS 84 e UTM</i></p>
        <p><i><a href="https://www.latlong.net/lat-long-utm.html" target="_blank">Convert Lat Long to UTM</a></i></p>        
    </div>
</div>

###  Análise das projeções

Quando usamos `Geopandas`, suas estruturas de dados `GeoSeries` e `GeoDataFrame` devem conter pelo menos uma coluna que indica, usando uma forma geométrica, sua posição geoespacial. Mas em muitos arquivos disponíveis, os dados geoespaciais para objetos são expressos com dois campos numéricos. Um para latitude e outro para longitude.

In [2]:
df_comisarias = pd.read_csv("../Data/comisarias.csv")
df_comisarias.head()

,X,Y,NOME
0,-58.468944,-34.683121,COMISARIA 52
1,-58.474649,-34.679169,COMISARIA 48
2,-58.501166,-34.661994,COMISARIA 42
3,-58.431981,-34.660395,COMISARIA 36
4,-58.402830,-34.641950,COMISARIA 4


Primeiro, devemos transformar os dados numéricos em uma nova coluna que identifica uma forma geométrica. Lembre-se de que a coluna é chamada por padrão de `geometry`. Nesse caso, a localização das delegacias é transformada em um ponto.

In [3]:
geo_comisarias = gpd.GeoDataFrame(df_comisarias, 
                                  geometry = gpd.points_from_xy(df_comisarias.X, 
                                                                df_comisarias.Y
                                                               )
                                 )
geo_comisarias.head()

,X,Y,NOME,geometry
0,-58.468944,-34.683121,COMISARIA 52,POINT (-58.46894 -34.68312)
1,-58.474649,-34.679169,COMISARIA 48,POINT (-58.47465 -34.67917)
2,-58.501166,-34.661994,COMISARIA 42,POINT (-58.50117 -34.66199)
3,-58.431981,-34.660395,COMISARIA 36,POINT (-58.43198 -34.66039)
4,-58.402830,-34.641950,COMISARIA 4,POINT (-58.40283 -34.64195)


In [4]:
#type(geo_comisarias)

Quando procuramos o CRS usado no ponto, nada aparece. Porque só transformamos os números `X` e `Y`, nas coordenadas da figura. Temos que indicar qual é o CRS explicitamente, com o método `crs`

In [5]:
geo_comisarias.crs

Vamos definir a projeção.

In [6]:
#geo_comisarias.crs = {'init' :'epsg:4326'} #deprecado
#geo_comisarias = geo_comisarias.set_crs("EPSG:4326")
geo_comisarias = geo_comisarias.set_crs(epsg = 4326)

In [7]:
geo_comisarias.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

###  Reprojeção

Às vezes, precisamos mudar o CRS que estamos usando, para transformar as coordenadas. Vamos levar as coordenadas que as delegacias possuem, WGS 84, por latitude e longitude (EPSG = 4326), para algumas coordenadas por UTM (EPSG = 32733).

In [8]:
 geo_comisarias.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
geo_comisarias.head()

,X,Y,NOME,geometry
0,-58.468944,-34.683121,COMISARIA 52,POINT (-58.46894 -34.68312)
1,-58.474649,-34.679169,COMISARIA 48,POINT (-58.47465 -34.67917)
2,-58.501166,-34.661994,COMISARIA 42,POINT (-58.50117 -34.66199)
3,-58.431981,-34.660395,COMISARIA 36,POINT (-58.43198 -34.66039)
4,-58.402830,-34.641950,COMISARIA 4,POINT (-58.40283 -34.64195)


Este comando emite um aviso. É um problema do `Geopandas` ao usar a biblioteca `pyproj`.

In [10]:
geo_comisarias = geo_comisarias.to_crs(epsg = 32733)

In [11]:
geo_comisarias.head()

,X,Y,NOME,geometry
0,-58.468944,-34.683121,COMISARIA 52,POINT (-6304210.347 2483981.461)
1,-58.474649,-34.679169,COMISARIA 48,POINT (-6305231.264 2483555.406)
2,-58.501166,-34.661994,COMISARIA 42,POINT (-6309786.922 2481474.256)
3,-58.431981,-34.660395,COMISARIA 36,POINT (-6305310.907 2490738.920)
4,-58.402830,-34.641950,COMISARIA 4,POINT (-6306256.037 2496114.174)


In [12]:
geo_comisarias.crs

<Projected CRS: EPSG:32733>
Name: WGS 84 / UTM zone 33S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, southern hemisphere between 80°S and equator, onshore and offshore. Angola. Congo. Democratic Republic of the Congo (Zaire). Gabon. Namibia. South Africa.
- bounds: (12.0, -80.0, 18.0, 0.0)
Coordinate Operation:
- name: UTM zone 33S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

---

### Referências

- [pyproj Documentation](https://pyproj4.github.io/pyproj/stable/index.html)

- [How Universal Transverse Mercator (UTM) Works](https://gisgeography.com/utm-universal-transverse-mercator-projection/)

- [EPSG Geodetic Parameter Dataset](https://epsg.org/home.html)